In [ ]:
# import argparse
import random
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from matplotlib.pyplot import imshow
from matplotlib import cm
import gc
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from PIL import Image
from tqdm import tqdm_notebook as tqdm

from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import ShuffleSplit
import random

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

iters = [10, 20, 30, 40, 50, 75, 100, 200, 500, 1000, None];

args = {
    "data_dir": "../data"
}


In [ ]:
os.listdir("../data/")

In [ ]:
train_labels = pd.read_csv('../data/train_labels.csv')

In [ ]:
DATA_DIR = '../data/train'
EXTENTION = '.tif'

num_train_labels = len(train_labels)

y = np.empty(num_train_labels)
X = None

for index, row in tqdm(train_labels.iterrows(), total=len(train_labels)):
    if (index % 20000 == 0):
        print("collecting")
        gc.collect();
    
    img_path = os.path.join(DATA_DIR, row["id"] + EXTENTION)
    
    img = Image.open(img_path).convert('1')
    width, height = img.size   # Get dimensions
    left = (width - 32)/2
    top = (height - 32)/2
    right = (width + 32)/2
    bottom = (height + 32)/2
    img = img.crop((left, top, right, bottom))#.resize((16, 16), Image.ANTIALIAS)
    arr = np.array(img).ravel()
    
    
    if X is None: 
        print("predicted mem usage: " + str((len(arr.ravel()) * num_train_labels * 4) /(2**30.0)) + " GB")
        X = np.empty((num_train_labels, len(arr.ravel())))
        imshow(np.asarray(img))
        print(img.size)
        
    y[index] = row["label"]
    X[index] = arr

gc.collect()

In [ ]:
preNormalized = X;
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=420)
X = None
preNormalized = None
gc.collect()

In [ ]:
clf = DecisionTreeClassifier()
score = cross_val_score(clf, X_train, y_train, cv=10, verbose=5, n_jobs=-1)
print();
print("SCORE FOR UNTOUCHED")
print(score)
print();

In [ ]:
#100
#100

#looking good
data = {}

for depth in iters:
    for leaves in iters:
        clf = DecisionTreeClassifier(max_depth=depth, max_leaf_nodes=leaves)
        score = cross_val_score(clf, X_train, y_train, cv=10, verbose=5, n_jobs=-1)
        print();
        print(score)
        print(depth)
        print(leaves)
        if (leaves not in data) data[leaves] = {};
        data[leaves][depth] = sum(score)/float(len(score))
        print()
        gc.collect()

In [ ]:
print(len(X))

In [ ]:
os.listdir('./')
X_plot = []
Y_plot = []
Z_plot = []
iters = [10, 20, 30, 40, 50, 75, 100, 200];
with open ("gridsearchresults.txt", "r") as myfile:
    rawString = myfile.readlines()
    counter = 5
    
    for depth in iters:
        for leaves in iters:
            X_plot.append(depth)
            Y_plot.append(leaves)
            currentArr = (rawString[counter] + rawString[counter + 1])[1:-2].split(" ")
            parsedArr = [float(x) for x in currentArr if x is not ""]
            Z_plot.append(sum(parsedArr)/float(len(parsedArr)))
            counter += 10

In [ ]:
X_plot = np.array(X_plot)
Z_plot = np.array(Z_plot)
Y_plot = np.array(Y_plot)

In [ ]:
fig = plt.figure(figsize=(8.0, 6.0))
ax = fig.gca(projection='3d')

# Make data.
X = X_plot
Y = Y_plot
# X, Y = np.meshgrid(X_plot, Y_plot)
Z= Z_plot

# Plot the surface.
surf = ax.scatter(X_plot, Y_plot, Z_plot)

# # Customize the z axis.
# ax.set_zlim(-1.01, 1.01)
# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
ax.set_xlabel("Depth Limit")
ax.set_ylabel("Child Node Limit")
ax.set_zlabel("Mean 10-fold score")

plt.savefig("gridSearch.png")
plt.show()

In [ ]:


param_dist = {
              "n_neighbors": sp_randint(2, 100),
              "p": [1, 2]
}
rand_kNN = RandomizedSearchCV(KNeighborsClassifier(), param_dist, n_jobs=-1, cv=10, n_iter=100, verbose=2)
rand_kNN.fit(X_train, y_train)

In [ ]:
pca = PCA(.4)
pca.fit(X_train)

In [ ]:
pca_train_img = pca.transform(X_train)

In [ ]:
print(len(X_train[0]))
print(len(pca_train_img[0]))
gc.collect()

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

In [ ]:
neigh.fit(pca_train_img, y_train)

In [ ]:
neigh.score(pca_train_img, y_train)

In [ ]:
param_dist = {
              "n_neighbors": sp_randint(2, 50),
              "p": [1, 2]
}
rand_kNN = RandomizedSearchCV(KNeighborsClassifier(), param_dist, n_jobs=-1, cv=10, n_iter=5, verbose=2)
rand_kNN.fit(pca_train_img, y_train)

In [ ]:
rand_kNN.best_score_

In [ ]:
rand_kNN.cv_results_

In [ ]:
kNN_last_score = cross_val_score(KNeighborsClassifier(n_neighbors=3, p=2), pca_train_img, y_train, cv=10, n_jobs=-1)

In [ ]:
kNN_last_score.mean()

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
xs = [];
ys = [];
zs = [];
for i in range(0, len(rand_kNN.cv_results_['params'])):
    xs.append(rand_kNN.cv_results_['params'][i]['n_neighbors'])
    ys.append(rand_kNN.cv_results_['params'][i]['p'])
    zs.append(rand_kNN.cv_results_['mean_test_score'][i])


xs.append(3)
ys.append(2)
zs.append(0.5784004227918418)

# print(xs, ys, zs)
    
# fig = plt.figure(figsize=(8, 6))

# ax = fig.add_subplot(111)

# ax.scatter(xs, zs, label=ys)


# # ax.set_zlabel('Mean 10-fold score')

# plt.savefig("gridSearchCredit")

scatter_x = np.array(xs)
scatter_y = np.array(zs)
group = np.array(ys)
cdict = {1: 'red', 2: 'blue'}

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
for g in np.unique(group):
    ix = np.where(group == g)
    ax.scatter(scatter_x[ix], scatter_y[ix], c = cdict[g], label = g, s = 100)
ax.set_xlabel('Number of Neighboors')
ax.set_ylabel('Mean 10-fold score')
ax.legend(title="Power of Minkowski Metric")
plt.show()



In [ ]:
best_dtc = DecisionTreeClassifier(max_depth=200, max_leaf_nodes=76)
adaboost = AdaBoostClassifier(base_estimator = best_dtc, n_estimators=100)

In [ ]:
adaboost_score = cross_val_score(adaboost, X_train, y_train, cv=10, verbose=10, n_jobs=-1)
print('adaboost Cross Validation Score', round(adaboost_score.mean() * 100, 2).astype(str) + '%')

In [ ]:
best_dtc2 = DecisionTreeClassifier(max_depth=200, max_leaf_nodes=200)
adaboost2 = AdaBoostClassifier(base_estimator = best_dtc2, n_estimators=10)

In [ ]:
adaboost_score2 = cross_val_score(adaboost2, X_train, y_train, cv=10, verbose=10, n_jobs=-1)
print('adaboost Cross Validation Score', round(adaboost_score2.mean() * 100, 2).astype(str) + '%')

In [ ]:
random.randint(1, 1000)

In [ ]:
using_10_estimators = [.6931273039438469, 0.6941568607646078, 0.6939352623339898, 0.6995101257512247, 0.7040347422107761, 0.6887340301974448, 0.6955006817148917, 0.6918648689592486, 0.7022372607444068, 0.700621180748447]

In [ ]:
sum(using_10_estimators)/float(len(using_10_estimators))

In [ ]:
ys = [sum(using_10_estimators)/float(len(using_10_estimators))]
ys.append(adaboost_score.mean())

In [ ]:
xs = [10, 100]

In [ ]:
plt.xlabel("Number of Estimators")
plt.ylabel("10-fold Mean Accuracy")
plt.scatter(xs, ys) #382 max

In [ ]:
param_dist = {
              "kernel": ["linear", "rbf", "sigmoid"],
              "C": [50, 100],
}

svm = GridSearchCV(SVC(), param_dist, cv=10, n_jobs=6, verbose=100)
svm.fit(X_train, y_train)


In [ ]:
svm.cv_results_

In [ ]:
resultsFinal = [
    [],
    [],
    [],
    []
];
print(len(X_train))
for i in range(1, 11):
    gc.collect()
    y_train_subset = y_train[:int((len(X_train)/10) * i)]
    X_train_subset = X_train[:int((len(X_train)/10) * i)]
    print(str(len(X_train)) + " " + str(len(y_train)))
    print(str(len(X_train_subset)) + " " + str(len(y_train_subset)))

    bestModels = [
        DecisionTreeClassifier(max_depth=200, max_leaf_nodes=200),
        KNeighborsClassifier(n_neighbors=48, p=2),
        AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=200, max_leaf_nodes=76), n_estimators=10),
        SVC(kernel="rbf", C=100)
    ]
    
    for j in range(0, len(bestModels)):
        start_time = time.time()
        bestModels[j].fit(X_train_subset, y_train_subset)
        predicSub = bestModels[j].predict(y_test)
        scores = [accuracy_score(original_ytest, predicAll)]
        resultsFinal[j].append(scores)
        print(str(i) + " " + str(scores) + "--- %s seconds ---" % (time.time() - start_time))
    

In [ ]:
# these were moved into the PCam CNN for viz work
print(resultsFinal)